In [1]:
# load data and write out sentence and target
import pandas as pd

loaded_set = pd.read_excel("Dataset/"+"training.xlsx")
#x_train, x_test,  y_train, y_test = train_test_split(loaded_set['Sentence'], loaded_set['MOS'], test_size=.2)
loaded_set['Sentence']

0      Weitergehende Sicherungsmaßnahmen können eine ...
1      In der großen Kasernenanlage im Norden Kiels k...
2      Premierminister David Lloyd George honorierte ...
3      Der Beitrag der Truppen dieser Dominions währe...
4      Eine Balance zwischen verschiedenen Lebensbere...
                             ...                        
895    Zwischen 1901 und 2010 ist er um ca 1,7 cm pro...
896    Aus Furcht vor einem Bürgerkrieg wollte sie – ...
897    In den meisten Politikfeldern gilt dafür seit ...
898    Aufgrund der Wärmekapazität des Gesteins, und ...
899    Die Klinge ist zumeist aus nicht rostfreiem Ko...
Name: Sentence, Length: 900, dtype: object

In [2]:
letter_count = []
avarange_letter_per_word = []
num_words = []
num_letters_array = []
longest_word_length = []
shortest_word_length = []
genitiv = []
akkusativ = []
dativ =[]

for sen in loaded_set['Sentence']:
    current_sen_split = sen.split()
    num_words.append(len(current_sen_split))
    num_letters = []
        
    if "des" in sen:
        genitiv.append(1)
    else:
        genitiv.append(0)

    if "dem" in sen:
        akkusativ.append(1)
    else:
        akkusativ.append(0)

    if "den" in sen:
        dativ.append(1)
    else:
        dativ.append(1)

    for y in range(len(current_sen_split)):
        current_word = current_sen_split[y]
   
        num_letters.append(len(current_word))
   
    current_lettercount = sum(num_letters)
    letter_count.append(current_lettercount) 
    avarange_letter_per_word.append(current_lettercount/len(current_sen_split))
    longest_word_length.append(max(num_letters)) 
    shortest_word_length.append(min(num_letters)) 
  



feature_dict = {
    'dativ':dativ, 
    'akkusativ': akkusativ, 
    'genitiv': genitiv, 
    'num_words':num_words,
    'letter_count':letter_count,
    'avarange_letter_per_word':avarange_letter_per_word,
    'longest_word_length':longest_word_length,
    'shortest_word_length':shortest_word_length, 
    }
feature_dataframe = pd.DataFrame(data=feature_dict)
feature_dataframe


,dativ,akkusativ,genitiv,num_words,letter_count,avarange_letter_per_word,longest_word_length,shortest_word_length
0,1,0,0,31,231,7.451613,22,2
1,1,0,0,11,61,5.545455,16,2
2,1,1,0,28,204,7.285714,20,2
3,1,0,0,24,139,5.791667,11,3
4,1,0,0,36,297,8.250000,23,2
...,...,...,...,...,...,...,...,...
895,1,0,0,27,114,4.222222,10,2
896,1,1,0,29,180,6.206897,14,1
897,1,1,0,43,285,6.627907,22,2
898,1,0,1,32,211,6.593750,15,3


In [3]:
from sklearn.model_selection import train_test_split

X = feature_dataframe
y = round(loaded_set['MOS'],2)

X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=.2)


In [84]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train[['num_words', 'longest_word_length', 'shortest_word_length', 'letter_count', 'avarange_letter_per_word']] = scaler.fit_transform(X_train[['num_words', 'longest_word_length', 'shortest_word_length', 'letter_count', 'avarange_letter_per_word']])

X_test[['num_words', 'longest_word_length', 'shortest_word_length', 'letter_count', 'avarange_letter_per_word']] = scaler.transform(X_test[['num_words', 'longest_word_length', 'shortest_word_length', 'letter_count', 'avarange_letter_per_word']])


tensorX = tf.constant(X_train.values)
tensorX = tf.reshape(tensorX [720, 8, 1])
tensorX.shape[0]

<ipython-input-84-80b552713886>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[['num_words', 'longest_word_length', 'shortest_word_length', 'letter_count', 'avarange_letter_per_word']] = scaler.fit_transform(X_train[['num_words', 'longest_word_length', 'shortest_word_length', 'letter_count', 'avarange_letter_per_word']])
C:\Anaconda3\envs\QandU\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
<i

InvalidArgumentError: Index out of range using input dim 2; input has only 2 dims [Op:StridedSlice] name: strided_slice/

In [88]:
import tensorflow as tf
n_timesteps =  tensorX.shape[0]
n_features =  tensorX.shape[1]
model = tf.keras.Sequential([
      #tf.keras.layers.GRU(100, input_shape=(n_timesteps, n_features), return_sequences=True, dropout=0.5),
      #tf.keras.layers.GRU(8),
      tf.keras.layers.InputLayer(8),
      #tf.keras.layers.GRU(4),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(1)
  ])

model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_68 (Dense)             (None, 64)                576       
_________________________________________________________________
dense_69 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_15 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_70 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 33        
Total params: 13,057
Trainable params: 13,057
Non-trainable params: 0
_________________________________________________________________


In [82]:
from keras.preprocessing.sequence import TimeseriesGenerator
n_input = 1 #how many samples/rows/timesteps to look in the past in order to forecast the next sample
n_features= X_train.shape[1] # how many predictors/Xs/features we have to predict y
b_size = 12# Number of timeseries samples in each batch



In [89]:
model.fit(tensorX ,y_train,epochs=500, verbose=1)

Epoch 261/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3864
Epoch 262/500
23/23 [==============================] - 0s 864us/step - loss: 0.3861
Epoch 263/500
23/23 [==============================] - 0s 773us/step - loss: 0.3805
Epoch 264/500
23/23 [==============================] - 0s 818us/step - loss: 0.3977
Epoch 265/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3894
Epoch 266/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3799
Epoch 267/500
23/23 [==============================] - 0s 909us/step - loss: 0.3682
Epoch 268/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3755
Epoch 269/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3814
Epoch 270/500
23/23 [==============================] - 0s 727us/step - loss: 0.3778
Epoch 271/500
23/23 [==============================] - 0s 773us/step - loss: 0.3816
Epoch 272/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3853
Epoc

In [90]:
import numpy as np
pred = model.predict(X_test)
rounded_pred = np.around(pred, decimals=2)
rounded_pred

array([[2.68],
       [1.87],
       [4.37],
       [2.68],
       [4.13],
       [4.22],
       [3.8 ],
       [2.64],
       [1.01],
       [2.32],
       [2.46],
       [3.84],
       [3.04],
       [1.92],
       [3.12],
       [3.82],
       [1.85],
       [2.92],
       [3.9 ],
       [5.21],
       [3.3 ],
       [2.16],
       [2.41],
       [3.74],
       [3.96],
       [4.2 ],
       [3.42],
       [3.04],
       [3.41],
       [4.06],
       [4.63],
       [3.36],
       [1.6 ],
       [4.47],
       [3.95],
       [1.74],
       [3.79],
       [1.55],
       [2.48],
       [4.86],
       [1.75],
       [1.74],
       [3.81],
       [2.8 ],
       [4.19],
       [2.06],
       [3.1 ],
       [4.32],
       [2.12],
       [2.54],
       [2.18],
       [4.78],
       [3.3 ],
       [2.63],
       [4.77],
       [2.53],
       [3.38],
       [2.04],
       [2.29],
       [3.05],
       [3.01],
       [4.03],
       [1.24],
       [2.44],
       [3.4 ],
       [4.05],
       [4.

In [91]:
def pearsonr(x, y):
  # Assume len(x) == len(y)
  n = len(x)
  sum_x = float(sum(x))
  sum_y = float(sum(y))
  sum_x_sq = sum(xi*xi for xi in x)
  sum_y_sq = sum(yi*yi for yi in y)
  psum = sum(xi*yi for xi, yi in zip(x, y))
  num = psum - (sum_x * sum_y/n)
  den = pow((sum_x_sq - pow(sum_x, 2) / n) * (sum_y_sq - pow(sum_y, 2) / n), 0.5)
  if den == 0: return 0
  return num / den
pearsonr(rounded_pred, y_test)

array([0.71567434], dtype=float32)